# Getting Started

Scryfall API:
"We kindly ask that you insert 50 – 100 milliseconds of delay between the requests you send to the server. (i.e., 10 requests per second on average)."

In [8]:
# import all the things
import numpy as np
import pandas as pd
import json, requests, pickle
from bs4 import BeautifulSoup

In [13]:
# Get setlist into dataframe
link = 'https://api.scryfall.com/sets'
response = requests.get(link)
sets = response.json()['data']
sets_df = pd.DataFrame(sets)
sets_df[['name','released_at']]

,name,released_at
0,Ultimate Box Topper,NaN
1,Ultimate Masters,2018-12-07
2,M19 Gift Pack,NaN
3,Game Night,2018-11-16
4,GRN Guild Kit,NaN
5,GRN Guild Kit Tokens,NaN
6,Guilds of Ravnica Promos,NaN
7,Guilds of Ravnica Tokens,NaN
8,Guilds of Ravnica,2018-10-05
9,Ravnica Weekend,NaN


## Clean up set data 

In [14]:
# Ignore sets with no release date (likely promotional / outliers), set release date to index
sets_df.dropna(subset=['released_at'], inplace=True)
sets_df.set_index('released_at', inplace=True)

# Remove repetitive information and online data
sets_df.drop(['digital','mtgo_code','parent_set_code','object'], axis=1, inplace=True)

In [24]:
# Make even cleaner for ease of exploration
clean_df = sets_df[['name','code','block_code','card_count','set_type']]
clean_df['set_type'].unique()

array(['masters', 'box', 'expansion', 'commander', 'memorabilia',
       'starter', 'core', 'duel_deck', 'spellbook', 'draft_innovation',
       'funny', 'from_the_vault', 'archenemy', 'planechase',
       'treasure_chest', 'premium_deck'], dtype=object)

In [44]:
# after extensive manual checking of set legality / onlineness
sets_to_drop = [
    'me1',
    'me2',
    'me3',
    'me4',
    'vma',
    'tpr',
    'e02',
    'gnt',
    'td0',
    'mgb',
    'ana',
    'w17',
    'w16',
    'itp'
]
types_to_drop = [
    'memorabilia',
    'funny',
    'treasure_chest'
]

In [10]:
sets_df[['name','code','set_type','card_count']][sets_df['set_type']=='box']

,name,code,set_type,card_count
3,Game Night,gnt,box,12
4,GRN Guild Kit,gk1,box,127
45,Explorers of Ixalan,e02,box,48
175,Modern Event Deck 2014,md1,box,26
278,Salvat 2011,ps11,box,224
284,Magic Online Theme Decks,td0,box,197
296,Duels of the Planeswalkers,dpa,box,113
381,Coldsnap Theme Decks,cst,box,62
388,Hachette UK,phuk,box,60
395,Salvat 2005,psal,box,720


In [ ]:
sets_df['set_type'].unique()